# Assignment 12

### 1.复习上课内容

### 2.回答以下理论问题

#### 1. 请写一下TF-IDF的计算公式

Ans: 
$$ TF = \frac{n(word)}{n(all)} $$
$$ TF = log\frac{n(word)}{n(all)} $$
$$ IDF = \frac{N}{n(d_w)} $$
$$ TF-IDF = TF * IDF $$
where is N is the number of documents, n(d_w) is the number of documents containing the word w.

#### 2. LDA算法的基本假设是什么？

Ans: The topics in a document obey a probability distribution as well as the words in each topic.

#### 3. 在TextRank算法中构建图的权重是如何得到的？

Ans: Using the cosine similarity as weight between two words after getting the corresponding word vector.

#### 4. 什么是命名实体识别？ 有什么应用场景？

Ans: It is to find each mention of a named entity in the text and label its type. It can be used in many conditions, such as common daily life including people and places, biomedical research like genes, and etc.

#### 5.NLP主要有哪几类任务 ？

Ans: NLP has two kinds of tasks including classification and generation tasks. The classification task is mainly applied to make classification such as sentiment, emotion; whereas the generation task is to generate text like machine translation, auto questioning, and text summerization, etc.

### 3.实践题

#### 3.1 手动实现TextRank算法 (在新闻数据中随机提取100条新闻训练词向量和做做法测试）

 提示：
 1. 确定窗口，建立图链接。   
 2. 通过词向量相似度确定图上边的权重
 3. 根据公式实现算法迭代(d=0.85)

In [1]:
import pandas as pd
import numpy as np
import gensim
import jieba
import re
# import stanfordcorenlp
import random
from gensim.models import word2vec
from gensim import models
import logging

In [2]:
CN_news = pd.read_csv('../5th/sqlResult_1558435.csv', encoding='gb18030')
CN_news.fillna('').head() # fill NA/Nan with ''

,id,author,source,content,feature,title,url
0,89617,,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [3]:
CN_news = CN_news.fillna('')

In [4]:
def cut(string):
    return ' '.join(jieba.cut(string, cut_all=False))

In [5]:
def token(string):
    return re.findall(r'[\d|\w]+', re.sub(r'\\n', '\n', string))

In [6]:
content = CN_news['content'].tolist()

In [7]:
# token(content[0])
# token(re.sub(r'\\n','\n','这是一个\\n测试2\n\n\n'))
# re.sub(r'\\n','\n','这是一个\\n测试2\n\n\n')
token('这是一个\\n测试2\n\n\n')

['这是一个', '测试2']

In [8]:
news_content = [token(i) for i in content]

In [9]:
news_content[52925]

['新华社照片',
 '马扎里沙里夫',
 '2017年5月6日',
 '一带一路',
 '好味道',
 '5',
 '阿富汗传统花式饺子',
 '4月21日',
 '在阿富汗北部城市马扎里沙里夫',
 '一名阿富汗妇女在家中制作传统花式饺子',
 '阿富汗传统花式饺子',
 '又称',
 '阿沙克',
 '是深受阿富汗人喜爱的一种传统美食',
 '这种花式饺子以面为皮',
 '用牛肉',
 '细香葱',
 '蒜末',
 '香菜',
 '番茄',
 '食用油及酸奶等制作馅料及蘸料',
 '新华社发',
 '科布拉',
 '阿克巴里摄']

In [10]:
# news_content = [j for i in news_content for j in i.split() if j != 'n']-------
news_content = [''.join(i) for i in news_content]

In [13]:
np.random.seed(9999)
random_index = np.random.randint(len(news_content),size=100)
random_index

array([75005, 22910, 25457, 33781, 26460, 54169, 17638, 64123,  4632,
       57978, 60236, 88552, 64673,  3502, 70689, 19296,  2686, 25679,
       23846, 36539, 16278,  2184, 67100,  6468, 77229, 26510, 35057,
       35883, 31369, 88778, 42643,  1950, 22227, 19289, 38667, 89505,
        2296,  3465, 16030, 69600,   173,  4766, 16165, 48688, 14137,
       70489, 81381, 25901, 84624, 54467, 14694, 52448, 80025, 61723,
       36216, 17033,  9478, 56340, 50314, 73846, 78153, 31578, 28552,
       47644, 72019, 50907, 86412, 52450, 59466, 58185, 52370, 38513,
       80218, 11243, 15008, 51426, 84452, 84523, 48554, 37990, 38938,
       70639, 57944, 52640, 74886, 23882, 20259, 67587, 22281, 81715,
       15742, 57824, 65069, 44215, 21657, 50768, 17880, 85508, 27441,
       62625])

In [14]:
random_news = np.array(news_content)[random_index].tolist()

In [15]:
len(random_news)

100

In [16]:
random_news = [cut(i) for i in random_news]

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/cm/kbmcn4j97tb1xc18l7kb__cr0000gn/T/jieba.cache
Loading model cost 0.720 seconds.
Prefix dict has been built succesfully.


In [17]:
random_news[1]

'新华社 照片 百色 广西 2017 年 4 月 10 日 透视 百色 说 脱贫 在 广西 百色市 右江区 四塘 镇 永靖 村 村民 驾驶 甘蔗 种植 机 种植 甘蔗 2016 年 3 月 17 日摄 自然 条件 差 传统 农业 无法 带动 贫困 群众 脱贫 该 怎么 去 打赢 脱贫 攻坚战 广西 百色 面对 的 困境 具有 典型 意义 然而 仅 去年 一年 百色 就 有 134 个 贫困村 脱贫 出列 这 是 如何 实现 的 推动 特色产业 发展 增强 贫困地区 自我 发展 能力 是 今年 政府 工作 报告 关于 扶贫 工作 的 重要 精神 之一 解析 百色 产业 扶贫 实践 为 老区 脱贫 攻坚 提供 了 一个 可 供 借鉴 的 样本 新华社 记者 黄孝邦 摄'

In [18]:
def get_stopwords():
    logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s', level=logging.INFO)
    # 加载停用词表
    stopword_set = set()
    with open("./stopwords-master/百度停用词表.txt", 'r', encoding="utf-8") as stopwords:
        for stopword in stopwords:
            stopword_set.add(stopword.strip("\n"))
    return stopword_set

In [19]:
## filter stop words
stopwords = get_stopwords()
# random_news_filter_SW = [j for i in random_news for j in i.split() if j not in stopwords]
# random_news_filter_SW = [' '.join(i) for i in random_news_filter_SW]
random_news_filter_SW = []
for i in range(len(random_news)):
    _line = random_news[i].split()
    _line_list = []
    for j in _line:
        if j not in stopwords:
            _line_list.append(j)
            
    random_news_filter_SW.append(_line_list)
    
random_news_filter_SW = [' '.join(i) for i in random_news_filter_SW]
random_news_filter_SW[1]

'新华社 照片 百色 广西 2017 年 4 月 10 日 透视 百色 说 脱贫 广西 百色市 右江区 四塘 镇 永靖 村 村民 驾驶 甘蔗 种植 机 种植 甘蔗 2016 年 3 月 17 日摄 自然 条件 差 传统 农业 带动 贫困 群众 脱贫 去 打赢 脱贫 攻坚战 广西 百色 面对 困境 典型 意义 仅 去年 一年 百色 134 贫困村 脱贫 出列 推动 特色产业 发展 增强 贫困地区 自我 发展 能力 政府 工作 报告 扶贫 工作 精神 解析 百色 产业 扶贫 实践 老区 脱贫 攻坚 提供 一个 供 借鉴 样本 新华社 记者 黄孝邦 摄'

In [20]:
random_news_filter_SW_list = [i.split() for i in random_news_filter_SW]

In [21]:
def training_word2vec():
    logging.basicConfig(format="%(asctime)s:%(levelname)s:%(message)s", level=logging.INFO)
#     sentences = word2vec.LineSentence()
    model = word2vec.Word2Vec(random_news_filter_SW_list, size=100, sg=1, min_count=1)
    model.save('./random_CN_news_wordVec.model')
    return model

model = training_word2vec()

2019-12-24 22:34:39,731:INFO:collecting all words and their counts
2019-12-24 22:34:39,732:INFO:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-12-24 22:34:39,736:INFO:collected 5366 word types from a corpus of 15324 raw words and 100 sentences
2019-12-24 22:34:39,737:INFO:Loading a fresh vocabulary
2019-12-24 22:34:39,750:INFO:effective_min_count=1 retains 5366 unique words (100% of original 5366, drops 0)
2019-12-24 22:34:39,751:INFO:effective_min_count=1 leaves 15324 word corpus (100% of original 15324, drops 0)
2019-12-24 22:34:39,767:INFO:deleting the raw counts dictionary of 5366 items
2019-12-24 22:34:39,768:INFO:sample=0.001 downsamples 29 most-common words
2019-12-24 22:34:39,768:INFO:downsampling leaves estimated 14158 word corpus (92.4% of prior 15324)
2019-12-24 22:34:39,778:INFO:estimated required memory for 5366 words and 100 dimensions: 6975800 bytes
2019-12-24 22:34:39,779:INFO:resetting layer weights
2019-12-24 22:34:39,843:INFO:training model wi

In [22]:
# model.wv.vocab['参考消息'].count
model.wv['新华社']

array([ 0.14318532, -0.10489807, -0.03794242, -0.02478698,  0.12174601,
       -0.08899361,  0.00642075, -0.08258772,  0.04967834,  0.08586293,
       -0.05007226,  0.07428997, -0.02547304, -0.1218941 , -0.15751004,
        0.10656352, -0.26908788, -0.15247628, -0.02440186, -0.02053007,
       -0.07679876,  0.01343962, -0.01850789,  0.00138471, -0.20141335,
       -0.09202338,  0.07886779,  0.13813326, -0.149757  , -0.02589867,
        0.00660518, -0.00123927,  0.1755541 , -0.09726332, -0.01428858,
       -0.13546106,  0.01132555, -0.07284861, -0.02151537, -0.12010807,
       -0.02966244, -0.08871925, -0.06960996,  0.08698625,  0.08608463,
        0.01245487, -0.11447557,  0.15003033, -0.08511181, -0.14613186,
        0.15533859,  0.1761068 , -0.07922481, -0.05772516, -0.07214306,
        0.06726139,  0.0236268 ,  0.01576287,  0.01086982, -0.0492137 ,
       -0.08515485, -0.094408  , -0.01475012,  0.02020209, -0.03440457,
       -0.11774959,  0.01135983,  0.10302687, -0.01994175,  0.10

In [23]:
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
def text_rank(inlist, wordvec, d=0.85, window_size = 3, n_iter = 10):
    def prepare_wordlink(): # sliding window
        word_link = defaultdict(list)
        for k in range(len(inlist)):
            news = inlist[k]
            for i in range(len(news) - window_size):
                word = news[i]
                for j in range(i+1, i + window_size):
    #                 window_item = inlist[j:(j+window_size)]

    #                 for k in range(1, len(window_item)):
                    next_word = news[j]
                    similarity = cosine_similarity(wordvec[word].reshape(1,-1), wordvec[next_word].reshape(1,-1))[0][0]
                    word_link[word].append([next_word, similarity])
                    word_link[next_word].append([word, similarity])

        return word_link
    
    def prepare_WS(Word_link):
        WS = defaultdict(float)
        for _key in Word_link:
            WS[_key] = float(1) / len(Word_link.keys())
            
        return WS
        
    word_link = prepare_wordlink()
#     print(word_link)
    WS = prepare_WS(word_link)
    
    ## start to realize text-rank
    all_words = word_link.keys()
    
    for _ in range(n_iter):
        for word in all_words:
            updated_WS = 1 - d

            for linked_word_list in word_link[word]:
                linked_word = linked_word_list[0]
                weight = linked_word_list[1]
                linked_word_WS = WS[linked_word]

                updated_WS += d * weight * linked_word_WS / sum([i[1] for i in word_link[linked_word]])

            WS[word] = updated_WS
            
#     WS = sorted(WS, key = lambda x: x.values)
    return WS
    
# word_WS = text_rank(random_news_filter_SW_list, model.wv)
# word_WS = text_rank([random_news_filter_SW_list[0]], model.wv)

In [24]:
# word_WS = text_rank([random_news_filter_SW_list[0]], model.wv)
# print(sorted(word_WS.items(), key = lambda x: x[1], reverse=True))

word_WS = text_rank(random_news_filter_SW_list, model.wv)
sorted(word_WS.items(), key = lambda x: x[1], reverse=True)

[('１', 47.41707965458946),
 ('０', 46.97771454523697),
 ('月', 43.82462258655011),
 ('日', 42.127514744791654),
 ('年', 35.537007414031365),
 ('２', 34.22207402033541),
 ('中国', 33.57589247831074),
 ('新华社', 23.87536954200026),
 ('中', 21.94903847625692),
 ('4', 18.905084956680714),
 ('４', 18.763838404323646),
 ('３', 18.755978628698735),
 ('2017', 18.144720339386257),
 ('５', 16.94002191854644),
 ('上', 16.804220703934813),
 ('说', 16.46600639791922),
 ('记者', 15.890231771895753),
 ('发展', 15.678048075606345),
 ('美国', 15.25305762582256),
 ('还', 14.384344619571124),
 ('直播', 13.926817042331127),
 ('６', 13.65369490905964),
 ('5', 13.625352827704399),
 ('７', 13.133705223423918),
 ('投资', 13.088254436766903),
 ('合作', 13.052923012616748),
 ('不', 12.499440353211899),
 ('经济', 12.034067620480128),
 ('比赛', 11.780933087797719),
 ('网络', 11.31715836577444),
 ('人', 11.055523142597822),
 ('企业', 10.738406605436012),
 ('都', 10.478063717710889),
 ('国家', 10.317626535268909),
 ('公司', 10.305001685372355),
 ('一个', 9.7656

### 3.2 使用词向量和k-means的方法寻找关键词

In [25]:
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

In [26]:
def total_wordvec(inlist, wordvec_model):
    vecword_dict = defaultdict(str)
    wordvec_summary = []
    for i in inlist:
        for j in i:
            vecword_dict[tuple(wordvec_model[j].tolist())] = j
    
    for _key in vecword_dict.keys():
        wordvec_summary.append(_key)
        
    return dict(vecword_dict), wordvec_summary

vec_to_word, total_wordvec = total_wordvec(random_news_filter_SW_list, model.wv)

In [27]:
# list(vec_to_word.items())[0]
len(total_wordvec)

5366

In [28]:
kmeans = KMeans(n_clusters=5)
kmeans.fit(total_wordvec)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [29]:
# kmeans.cluster_centers_
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, total_wordvec)
closest
kmeans.labels_[0:10]

array([4, 3, 0, 0, 1, 4, 1, 4, 3, 4], dtype=int32)

In [30]:
## get key words with the most n nearest distance to the cluster center
def KMeans_keywords(model, data, N_nearest):
    data2 = data[:]
    keywords = []
    for _ in range(N_nearest):
        closest_index, _ = pairwise_distances_argmin_min(model.cluster_centers_, data2)
#         print(closest_index)
        closest_words = [vec_to_word[tuple(x)] for x in np.array(data2)[closest_index].tolist()]
        keywords += closest_words
#         data2.pop(closest_index.tolist())
#         del_item = np.array(data2)[closest_index].tolist()
#         del del_item
#         print(len(data2))
#         for i in closest_index.tolist():
#             del data2[i]
        data2 = np.delete(np.array(data2), closest_index, 0)#.tolist()
        
    return keywords

Keywords = KMeans_keywords(kmeans, total_wordvec, 5)
Keywords

['格局',
 '全国',
 '找到',
 '原因',
 '人',
 '期限内',
 '称',
 '系列',
 '洛杉矶',
 '经济',
 '上海市委',
 '当地',
 '影响',
 '真的',
 '说',
 '西红柿',
 '庆祝',
 '集团',
 '示范',
 '一路',
 '成果',
 '上海',
 '旅游',
 '努力',
 '投资']

In [31]:
a = [1, 2, 3]
# np.array(a)[np.array([1,2])]
# # a.pop([1,2]) # wrong
np.delete(np.array(a),[np.array([0,2])])
a
# np.array(a)
# a
# b = a
# b = a[:]
# b.append(4)
# a, b
# np.array(total_wordvec)[np.array([3725, 1478, 1331, 3547,  895, 3918,  604, 1302, 1824, 1231])].tolist()

[1, 2, 3]

提示：   
1.使用3.1训练好的词向量   
2.可使用sklearn等机器学习库

### 3.3  提取新闻人物里的对话。(使用以上提取小数据即可）

提示：    
1.寻找预料里具有表示说的意思。    
2.使用语法分析提取句子结构。    
3.检测谓语是否有表示说的意思。

In [32]:
## filter stop words
stopwords = get_stopwords()
# random_news_filter_SW = [j for i in random_news for j in i.split() if j not in stopwords]
# random_news_filter_SW = [' '.join(i) for i in random_news_filter_SW]
news_filter_SW = []
news_content2 = [cut(i) for i in news_content]
for i in range(len(news_content2)):
    _line = news_content2[i].split()
    _line_list = []
    for j in _line:
        if j not in stopwords:
            _line_list.append(j)
            
    news_filter_SW.append(_line_list)
    
news_filter_SW = [' '.join(i) for i in news_filter_SW]
news_filter_SW[0]

'本周 6 月 12 日起 小米 手机 6 15 款 机型 外 机型 已 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 确保 工程师 精力 系统优化 工作 有人 猜测 精力 用到 MIUI9 研发 之中 MIUI8 去年 5 月 发布 距今已有 一年 有余 更新换代 MIUI9 确切 信息 等待 官方消息'

In [33]:
news_filter_SW_list = [i.split() for i in news_filter_SW]

In [34]:
def training_word2vec():
    logging.basicConfig(format="%(asctime)s:%(levelname)s:%(message)s", level=logging.INFO)
#     sentences = word2vec.LineSentence()
    model = word2vec.Word2Vec(news_filter_SW_list, size=100, sg=0, min_count=5)
#     model.save('./CN_news_wordVec.model')
    model.save('./CN_news_wordVec.model')
    return model

model_all = training_word2vec()

2019-12-24 22:37:36,969:INFO:collecting all words and their counts
2019-12-24 22:37:36,971:INFO:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-12-24 22:37:37,535:INFO:PROGRESS: at sentence #10000, processed 2656448 words, keeping 147764 word types
2019-12-24 22:37:37,813:INFO:PROGRESS: at sentence #20000, processed 3932532 words, keeping 173188 word types
2019-12-24 22:37:38,087:INFO:PROGRESS: at sentence #30000, processed 5164998 words, keeping 188669 word types
2019-12-24 22:37:38,385:INFO:PROGRESS: at sentence #40000, processed 6453691 words, keeping 205055 word types
2019-12-24 22:37:38,712:INFO:PROGRESS: at sentence #50000, processed 7869034 words, keeping 221319 word types
2019-12-24 22:37:39,035:INFO:PROGRESS: at sentence #60000, processed 9316575 words, keeping 236610 word types
2019-12-24 22:37:39,329:INFO:PROGRESS: at sentence #70000, processed 10627418 words, keeping 248986 word types
2019-12-24 22:37:39,633:INFO:PROGRESS: at sentence #80000, processe

2019-12-24 22:38:29,184:INFO:worker thread finished; awaiting finish of 0 more threads
2019-12-24 22:38:29,185:INFO:EPOCH - 4 : training on 13258895 raw words (12194157 effective words) took 11.7s, 1045361 effective words/s
2019-12-24 22:38:30,193:INFO:EPOCH 5 - PROGRESS: at 4.08% examples, 1072882 words/s, in_qsize 5, out_qsize 0
2019-12-24 22:38:31,200:INFO:EPOCH 5 - PROGRESS: at 8.01% examples, 1057658 words/s, in_qsize 5, out_qsize 0
2019-12-24 22:38:32,214:INFO:EPOCH 5 - PROGRESS: at 16.86% examples, 1027857 words/s, in_qsize 6, out_qsize 0
2019-12-24 22:38:33,218:INFO:EPOCH 5 - PROGRESS: at 27.22% examples, 1029765 words/s, in_qsize 5, out_qsize 0
2019-12-24 22:38:34,222:INFO:EPOCH 5 - PROGRESS: at 37.60% examples, 1035942 words/s, in_qsize 5, out_qsize 0
2019-12-24 22:38:35,230:INFO:EPOCH 5 - PROGRESS: at 47.00% examples, 1035497 words/s, in_qsize 5, out_qsize 0
2019-12-24 22:38:36,238:INFO:EPOCH 5 - PROGRESS: at 56.19% examples, 1036686 words/s, in_qsize 5, out_qsize 0
2019-12-

In [35]:
all_words = list(model_all.wv.vocab.keys())
# print(list(all_words)[:10])

def similar_meaning_of_saying(target_word, n_similarity = 20, cutoff = 0.5):
    wordvec_of_say = model_all.wv[target_word] #'说'
    word_similarity = defaultdict(list)

    for i in all_words:
        word_similarity[i] = [i, cosine_similarity(wordvec_of_say.reshape(1,-1), model_all.wv[i].reshape(1,-1))[0][0]]
        
    word_similarity = sorted(dict(word_similarity).values(), key = lambda x: x[1], reverse=True)
    
    similar_words = [i[0] for i in word_similarity if i[1] >= cutoff]
    if len(similar_words) > 20: return similar_words[:n_similarity]
    else: return similar_words
        

# similar_words
similar_words = similar_meaning_of_saying('说')
# The result means that the model trained with little data is not good at all, so I use model trained with all data instead.

In [36]:
total_say_words = []
for i in similar_words:
    total_say_words += similar_meaning_of_saying(i, n_similarity=5)
#     print(total_say_words)
    total_say_words = list(set(total_say_words))

print(total_say_words)

['提到', '解读', '副处长', '司', '营销部', '据介绍', '张伟', '知情', '刘', '文说', '主任', '海涛', '感激', '阿什顿', '肯定', '中说', '关强', '沙阿', '副局长', '指出', '透露', '不解', '常务副', '说道', '介绍', '特别强调', '副', '明说', '司长', '建议', '发推', '说据', '北京大学国际关系学院', '莫雷尔', '称该', '发表', '感慨', '告诉', '呼吁', '中称', '部副', '说完', '口中', '地说', '直言', '管理司', '普遍认为', '对此', '称', '提及', '简短', '确实', '说', '提出', '陆', '坦言', '言', '还称', '卡明斯']


In [37]:
cosine_similarity(model_all.wv['介绍'].reshape(1,-1), model_all.wv['说'].reshape(1,-1))

array([[0.64233553]], dtype=float32)

## Test pyltp

In [38]:
import pyltp
from gensim.models import Word2Vec
from pyltp import Segmentor
import jieba
from gensim.models.word2vec import LineSentence
from pyltp import  SentenceSplitter,NamedEntityRecognizer,Postagger,Parser,Segmentor
from gensim import models
import numpy as np

In [39]:
cws_model = "ltp_data_v3.4.0/cws.model"
pos_model = "ltp_data_v3.4.0/pos.model"
par_model = "ltp_data_v3.4.0/parser.model"
ner_model = "ltp_data_v3.4.0/ner.model"

In [40]:
def get_word_list(sentence, model):
    #得到分词
    segmentor = Segmentor()
    segmentor.load(model)
    word_list = list(segmentor.segment(sentence))
    segmentor.release()
    return word_list

def get_postag_list(word_list, model):
    #得到词性标注
    postag = Postagger()
    postag.load(model)
    postag_list = list(postag.postag(word_list))
    postag.release()
    return postag_list

def get_parser_list(word_list, postag_list, model):
    #得到依存关系
    parser = Parser()
    parser.load(model)
    arcs = parser.parse(word_list, postag_list)
    arc_list = [(arc.head, arc.relation) for arc in arcs]
    parser.release()
    return arc_list

In [41]:
random_news_line = ''.join([''.join(i.split()) for i in random_news])
random_news_line[:500]
# random_news_raw_line = ''.join([i for i in np.array(content)[random_index].tolist()])
# random_news_raw_line[:500]

'新华社照片黄金海岸澳大利亚2017年5月25日体育32羽毛球苏迪曼杯中国台北对阵韩国5月25日韩国队选手孙完虎在男单比赛中他以2比1战胜中国台北队选手周天成当日在澳大利亚黄金海岸举行的2017年苏迪曼杯世界羽毛球混合团体锦标赛四分之一决赛中中国台北队对阵韩国队新华社记者吕小炜摄新华社照片百色广西2017年4月10日透视百色说脱贫在广西百色市右江区四塘镇永靖村村民驾驶甘蔗种植机种植甘蔗2016年3月17日摄自然条件差传统农业无法带动贫困群众脱贫该怎么去打赢脱贫攻坚战广西百色面对的困境具有典型意义然而仅去年一年百色就有134个贫困村脱贫出列这是如何实现的推动特色产业发展增强贫困地区自我发展能力是今年政府工作报告关于扶贫工作的重要精神之一解析百色产业扶贫实践为老区脱贫攻坚提供了一个可供借鉴的样本新华社记者黄孝邦摄新华社长沙4月12日电记者阳建11日中车株洲电力机车有限公司与马来西亚交通部在长沙签订两款米轨1米宽轨距动车组购销合同包括13列混合动力电动车组和9列现代超级动车组这是自2014年以来中国再次获得马来西亚的米轨动车组订单据中车株洲电力机车有限公司动车组研发部部长周安德介绍13列混合动'

In [42]:
word_list = get_word_list(random_news_line, cws_model)
# word_list = get_word_list(random_news_raw_line, cws_model)

In [43]:
print(word_list[:10])
len(word_list)

['新华社', '照片', '黄金', '海岸', '澳大利亚', '2017年', '5月', '25日', '体育', '32']


19885

In [44]:
postag_list = get_postag_list(word_list, pos_model)

In [45]:
parser_list = get_parser_list(word_list, postag_list, par_model)

In [46]:
structure_summary = []
for i in range(len(word_list)):
    structure_summary.append([word_list[i], postag_list[i], parser_list[i]])
    print(word_list[i], postag_list[i], parser_list[i])

新华社 ni (2, 'ATT')
照片 n (21, 'ATT')
黄金 n (4, 'ATT')
海岸 n (5, 'ATT')
澳大利亚 ns (21, 'ATT')
2017年 nt (7, 'ATT')
5月 nt (8, 'ATT')
25日 nt (21, 'ATT')
体育 n (12, 'ATT')
32 m (11, 'ATT')
羽毛球 n (12, 'ATT')
苏迪曼杯 nz (21, 'ATT')
中国 ns (14, 'ATT')
台北 ns (21, 'ATT')
对阵 v (21, 'ATT')
韩国 ns (20, 'ATT')
5月 nt (18, 'ATT')
25日 nt (20, 'ATT')
韩国队 ni (20, 'ATT')
选手 n (21, 'ATT')
孙完虎 nh (31, 'SBV')
在 p (31, 'ADV')
男单 j (24, 'ATT')
比赛 v (25, 'ATT')
中 nd (22, 'POB')
他 r (31, 'SBV')
以 p (31, 'ADV')
2 m (29, 'ADV')
比 v (27, 'POB')
1 m (29, 'VOB')
战胜 v (0, 'HED')
中国 ns (33, 'ATT')
台北队 ni (34, 'ATT')
选手 n (35, 'ATT')
周天成 nh (41, 'SBV')
当日 nt (41, 'ADV')
在 p (41, 'ADV')
澳大利亚 ns (40, 'ATT')
黄金 n (40, 'ATT')
海岸 n (37, 'POB')
举行 v (51, 'ATT')
的 u (41, 'RAD')
2017年 nt (49, 'ATT')
苏迪曼杯 nz (49, 'ATT')
世界 n (49, 'ATT')
羽毛球 n (48, 'ATT')
混合 v (48, 'ATT')
团体 n (49, 'ATT')
锦标赛 n (51, 'ATT')
四分之一 m (51, 'ATT')
决赛 v (52, 'ATT')
中 nd (55, 'ADV')
中国 ns (54, 'ATT')
台北队 ni (55, 'SBV')
对阵 v (31, 'COO')
韩国队 ni (55, 'VOB')
新华社 ni (58,

人体 n (614, 'SBV')
所 u (614, 'LAD')
受 v (616, 'ATT')
辐射 v (616, 'ATT')
水平 n (617, 'SBV')
相当 v (620, 'ADV')
于 p (617, 'CMP')
频繁 a (618, 'POB')
做 v (603, 'VOB')
全身 n (622, 'ATT')
Ｃ ws (623, 'ATT')
Ｔ ws (624, 'SBV')
扫描 v (620, 'VOB')
远 a (628, 'ATT')
超 v (628, 'ATT')
正常 a (628, 'ATT')
水平 n (632, 'ATT')
这样 r (632, 'ATT')
的 u (629, 'RAD')
辐射 v (632, 'ATT')
强度 n (624, 'VOB')
增加 v (624, 'COO')
了 u (633, 'RAD')
人 n (636, 'SBV')
患 v (639, 'ATT')
癌症 n (636, 'VOB')
的 u (636, 'RAD')
风险 n (641, 'FOB')
如何 r (641, 'ADV')
解决 v (633, 'COO')
这个 r (643, 'ATT')
难题 n (648, 'ATT')
美国 ns (646, 'ATT')
康奈尔 nz (646, 'ATT')
大学 n (647, 'ATT')
生物学家 n (648, 'ATT')
克里斯托弗梅森 nh (651, 'SBV')
把 p (651, 'ADV')
目光 n (649, 'POB')
放 v (641, 'COO')
在 p (651, 'CMP')
大象 n (654, 'ATT')
身上 nl (652, 'POB')
梅森 nh (656, 'SBV')
说 v (651, 'COO')
大象 n (658, 'SBV')
患 v (661, 'ATT')
癌症 n (658, 'VOB')
的 u (658, 'RAD')
风险 n (662, 'SBV')
低 a (664, 'SBV')
而且 c (664, 'ADV')
有 v (656, 'VOB')
大量 m (668, 'ATT')
p53 m (667, 'ATT')
基因 n (668, 'ATT

暨 c (1113, 'LAD')
颁奖 v (1110, 'COO')
活动 v (1104, 'VOB')
据 p (1104, 'COO')
了解 v (1115, 'POB')
这 r (1118, 'SBV')
是 v (1116, 'VOB')
上海 ns (1123, 'SBV')
连续 a (1122, 'ATT')
第二 m (1122, 'ATT')
年 q (1123, 'ADV')
举办 v (1118, 'VOB')
此类 r (1125, 'ATT')
大赛 v (1123, 'VOB')
除 p (1139, 'ADV')
在 p (1129, 'ADV')
上海 ns (1127, 'POB')
举办 v (1132, 'ATT')
的 u (1129, 'RAD')
启动 v (1132, 'ATT')
仪式 n (1133, 'ATT')
外 nd (1134, 'ATT')
５月 nt (1135, 'ATT')
２３日 nt (1139, 'ADV')
主办方 n (1139, 'SBV')
还 d (1139, 'ADV')
将 d (1139, 'ADV')
赴 v (1123, 'COO')
台北 ns (1139, 'VOB')
举办 v (1139, 'COO')
发布会 n (1144, 'ATT')
完原 v (1144, 'ATT')
标题 n (1145, 'ATT')
投资人 n (1147, 'SBV')
实名 v (1147, 'ADV')
举报 v (1141, 'COO')
四三九九带病 d (1149, 'ADV')
闯 v (1147, 'COO')
关 n (1149, 'VOB')
IPO ws (1152, 'SBV')
称 v (1149, 'COO')
大 a (1154, 'ATT')
股东 n (1155, 'ATT')
蔡文胜 nh (1156, 'SBV')
偷 v (1152, 'VOB')
逃 v (1156, 'COO')
税款 n (1161, 'ATT')
数亿 m (1160, 'ATT')
元 q (1161, 'ATT')
作者 n (1162, 'ATT')
赵敏康 nh (1165, 'SBV')
6月 nt (1164, 'ATT')
20日 nt (11

10000万 m (1613, 'ATT')
资金系 n (1614, 'ATT')
邵晓舒 nh (1615, 'SBV')
挪用 v (1620, 'ATT')
上海 ns (1617, 'ATT')
鼎麟 nz (1615, 'VOB')
等 u (1617, 'RAD')
的 u (1615, 'RAD')
资金 n (1602, 'VOB')
而 c (1622, 'ADV')
来 v (1626, 'ATT')
的 u (1622, 'RAD')
4 m (1626, 'ATT')
西藏 ns (1626, 'ATT')
晨麒系 n (1627, 'ATT')
邵晓舒 nh (1628, 'SBV')
控制 v (1630, 'ATT')
的 u (1628, 'RAD')
企业 n (1631, 'ATT')
邵晓舒 nh (1634, 'SBV')
以 p (1634, 'ADV')
叶雨明 nh (1632, 'POB')
为 v (1565, 'COO')
马甲 n (1634, 'VOB')
参与 v (1653, 'SBV')
4399 m (1638, 'ATT')
游族 nz (1639, 'ATT')
网络 n (1641, 'ATT')
的 u (1639, 'RAD')
管理 v (1653, 'SBV')
对 p (1653, 'ADV')
其 r (1644, 'ATT')
经营 v (1642, 'POB')
产生 v (1642, 'POB')
重大 a (1647, 'ATT')
影响 v (1645, 'VOB')
这样 r (1650, 'ATT')
的 u (1648, 'RAD')
交易 n (1647, 'VOB')
怎么 r (1653, 'ADV')
不 d (1653, 'ADV')
是 v (1634, 'COO')
关联 v (1663, 'FOB')
交易 v (1658, 'SBV')
这样 r (1658, 'ATT')
的 u (1656, 'RAD')
关联 v (1654, 'VOB')
交易 v (1658, 'VOB')
怎么 r (1661, 'ADV')
能 v (1663, 'ADV')
不 d (1663, 'ADV')
审查 v (1653, 'VOB')
西藏 ns (166

竞选 v (2113, 'ATT')
中 nd (2114, 'ADV')
承诺 v (2116, 'ATT')
的 u (2114, 'RAD')
退出 v (2111, 'POB')
巴黎 ns (2118, 'ATT')
气候 n (2119, 'ATT')
协定 n (2116, 'VOB')
付诸实施 i (2099, 'VOB')
这样一来特朗普 r (2122, 'ATT')
先生 n (2128, 'SBV')
就 p (2128, 'ADV')
两 m (2125, 'ATT')
次 q (2128, 'ADV')
给 p (2128, 'ADV')
美国 ns (2126, 'POB')
造成 v (2120, 'COO')
了 u (2128, 'RAD')
巨大 a (2131, 'ATT')
损失 n (2128, 'VOB')
使 v (2128, 'COO')
他 r (2135, 'ATT')
的 u (2133, 'RAD')
国家 n (2132, 'DBL')
孤立 v (2132, 'VOB')
于 p (2136, 'CMP')
自己 r (2141, 'ATT')
的 u (2138, 'RAD')
伙伴 n (2141, 'ATT')
文章 n (2137, 'POB')
称 v (2132, 'COO')
没有 v (2142, 'VOB')
哪个 r (2145, 'ATT')
国家 n (2143, 'DBL')
比 p (2154, 'ADV')
中国 ns (2146, 'POB')
更 d (2149, 'ADV')
能 v (2154, 'ADV')
从 p (2154, 'ADV')
这种 r (2152, 'ATT')
状况 n (2153, 'ATT')
中 nd (2150, 'POB')
获益 v (2143, 'VOB')
中国 ns (2156, 'ATT')
领导人 n (2158, 'SBV')
已经 d (2158, 'ADV')
晋升 v (2154, 'VOB')
为 v (2163, 'ATT')
自由 a (2161, 'ATT')
贸易 v (2159, 'VOB')
的 u (2161, 'RAD')
庇护者 n (2168, 'SBV')
在 p (2168, 'ADV')

...

税法 n (17637, 'ATT')
的 u (17635, 'RAD')
主动 a (17638, 'ADV')
遵守 v (17634, 'POB')
蒋震说 nh (17640, 'SBV')
实施 v (17638, 'COO')
综合 v (17645, 'ADV')
和 c (17643, 'LAD')
分类 v (17641, 'COO')
相 d (17645, 'ADV')
结合 v (17647, 'ATT')
的 u (17645, 'RAD')
个人所得税 n (17648, 'FOB')
改革 v (17640, 'VOB')
需要 v (17640, 'VOB')
构建 v (17649, 'VOB')
新 a (17655, 'ATT')
的 u (17651, 'RAD')
税收 n (17654, 'FOB')
征管 j (17655, 'ATT')
制度 n (17650, 'VOB')
在 p (17660, 'ADV')
张连起 nh (17656, 'POB')
看来 v (17660, 'ADV')
要 v (17660, 'ADV')
加快 v (17650, 'COO')
修订 v (17664, 'ATT')
税收 n (17663, 'FOB')
征管 v (17664, 'ATT')
法 n (17665, 'SBV')
建立 v (17660, 'VOB')
个人 n (17667, 'ATT')
收入 n (17672, 'SBV')
和 c (17671, 'LAD')
财产 n (17670, 'ATT')
信息 n (17671, 'ATT')
系统 n (17667, 'COO')
推动 v (17665, 'COO')
现金 n (17674, 'ATT')
交易 v (17681, 'ATT')
规范 a (17681, 'ATT')
管理 v (17675, 'COO')
等 u (17675, 'RAD')
社会 n (17681, 'ATT')
综合 v (17681, 'ATT')
配套 a (17681, 'ATT')
条件 n (17684, 'SBV')
的 u (17681, 'RAD')
逐步 d (17684, 'ADV')
完善 v (17672, 'VOB')
为 p (

联赛 n (18227, 'ATT')
特步 d (18227, 'ATT')
CUFL ws (18227, 'ATT')
大足 n (18223, 'ATT')
联赛 n (18225, 'ATT')
校园 n (18225, 'ATT')
组 n (18227, 'ATT')
西北 nl (18227, 'ATT')
赛区 n (18228, 'ATT')
决赛 v (18233, 'SBV')
在 p (18233, 'ADV')
西安 ns (18232, 'ATT')
外国语 n (18232, 'ATT')
大学 n (18229, 'POB')
开幕 v (18201, 'COO')
在 p (18264, 'ADV')
揭幕战 n (18236, 'ATT')
中 nd (18239, 'ATT')
西安 ns (18239, 'ATT')
石油 n (18239, 'ATT')
大学 n (18234, 'POB')
以 p (18264, 'ADV')
7 m (18244, 'ADV')
比 p (18244, 'ADV')
1 m (18242, 'POB')
战胜 v (18240, 'POB')
宁夏 ns (18247, 'ATT')
医科 n (18247, 'ATT')
大学 n (18244, 'VOB')
来自 v (18262, 'ATT')
新疆 ns (18250, 'ATT')
内蒙古 ns (18251, 'ATT')
青海 ns (18252, 'ATT')
西藏 ns (18253, 'ATT')
陕西 ns (18254, 'ATT')
山西 ns (18255, 'ATT')
宁夏 ns (18262, 'ATT')
和 c (18257, 'LAD')
甘肃 ns (18255, 'COO')
的 u (18255, 'RAD')
16 m (18260, 'ATT')
支 q (18262, 'ATT')
高校 j (18262, 'ATT')
球队 n (18240, 'POB')
将 d (18264, 'ADV')
争夺 v (18233, 'COO')
4 m (18266, 'ATT')
个 q (18268, 'ATT')
校园 n (18268, 'ATT')
组 n (18270, 'AT

相继 d (18811, 'ADV')
受 v (18797, 'COO')
邀 v (18815, 'ATT')
访 v (18815, 'ATT')
巴 j (18813, 'VOB')
演出 v (18811, 'VOB')
均 d (18817, 'ADV')
取得 v (18811, 'COO')
了 u (18817, 'RAD')
圆满 a (18831, 'ADV')
成功 a (18819, 'COO')
目前 nt (18831, 'ADV')
中巴 n (18823, 'ATT')
文化 n (18825, 'ATT')
主管 v (18825, 'ATT')
部门 n (18831, 'SBV')
还 d (18831, 'ADV')
正 d (18831, 'ADV')
就 d (18831, 'ADV')
在 p (18831, 'ADV')
巴林 ns (18829, 'POB')
设立 v (18817, 'COO')
中国 ns (18834, 'ATT')
文化 n (18834, 'ATT')
中心 n (18836, 'ATT')
具体 a (18836, 'ATT')
事宜 n (18831, 'VOB')
加强 v (18831, 'COO')
工作 v (18839, 'ATT')
层面 n (18840, 'SBV')
沟通 v (18837, 'VOB')
与 c (18842, 'LAD')
协作 v (18840, 'COO')
以 p (18848, 'ADV')
共同 d (18845, 'ADV')
推动 v (18843, 'POB')
中心 n (18845, 'DBL')
尽早 d (18848, 'ADV')
建成 v (18840, 'COO')
我 r (18850, 'SBV')
相信 v (18848, 'COO')
在 p (18850, 'VOB')
一带 n (18853, 'ATT')
一路 m (18855, 'ATT')
的 u (18853, 'RAD')
带动 v (18851, 'POB')
下 nd (18857, 'ATT')
中巴 n (18861, 'ATT')
全方位 n (18861, 'ATT')
友好 a (18860, 'ADV')
合作 v (18861

债务 n (19313, 'SBV')
付息 v (19314, 'ATT')
支出 n (19315, 'SBV')
超过 v (19305, 'VOB')
当年 nt (19327, 'ATT')
一般 a (19327, 'ATT')
公共 b (19319, 'ATT')
预算 n (19320, 'ATT')
支出 n (19327, 'ATT')
１０ m (19327, 'ATT')
的 u (19321, 'RAD')
或者 c (19326, 'LAD')
专项 b (19325, 'ATT')
债务 n (19326, 'SBV')
付息 v (19327, 'ATT')
支出 n (19328, 'SBV')
超过 v (19315, 'VOB')
当年 nt (19337, 'ADV')
政府性 n (19331, 'ATT')
基金 n (19332, 'ATT')
预算 n (19334, 'ATT')
支出 v (19334, 'ATT')
１０ m (19337, 'ATT')
的 u (19334, 'RAD')
必须 d (19337, 'ADV')
启动 v (19328, 'COO')
财政 n (19339, 'ATT')
重整 v (19340, 'ATT')
计划 n (19341, 'ATT')
预案 n (19337, 'VOB')
指出 v (19337, 'COO')
政府性 n (19344, 'ATT')
债务 n (19345, 'ATT')
风险 n (19346, 'ATT')
事件 n (19347, 'SBV')
是 v (19342, 'VOB')
指 v (19347, 'VOB')
政府 n (19355, 'SBV')
已经 d (19355, 'ADV')
或者 c (19355, 'ADV')
可能 v (19355, 'ADV')
无法 v (19355, 'ADV')
按期 d (19355, 'ADV')
支付 v (19348, 'VOB')
政府 n (19357, 'ATT')
债务 n (19358, 'ATT')
本息 n (19361, 'FOB')
或者 c (19361, 'ADV')
无力 v (19361, 'ADV')
履行 v (19355, 'COO')


以 p (19881, 'ADV')
5 m (19872, 'ADV')
比 p (19872, 'ADV')
3 m (19870, 'POB')
战胜 v (19868, 'POB')
纳什维尔捕食者队 ni (19874, 'ATT')
双方 n (19876, 'ATT')
总 b (19876, 'ATT')
比分 n (19872, 'VOB')
以 p (19879, 'ADV')
1 m (19879, 'ADV')
比 v (19881, 'ADV')
1 m (19879, 'VOB')
战平 v (19848, 'COO')
新华社 ni (19883, 'SBV')
发 v (19881, 'COO')
赵汉荣 nh (19883, 'VOB')
摄 v (19883, 'COO')


In [47]:
say_structure_summary = [i for i in structure_summary if i[0] in similar_words]

In [49]:
print(say_structure_summary)
len(say_structure_summary)

[['说', 'v', (68, 'COO')], ['介绍', 'v', (230, 'POB')], ['说', 'v', (651, 'COO')], ['说', 'v', (706, 'COO')], ['说', 'v', (766, 'COO')], ['称', 'v', (1149, 'COO')], ['称', 'v', (1243, 'COO')], ['称', 'v', (1300, 'COO')], ['称', 'v', (1421, 'COO')], ['称', 'v', (1994, 'ATT')], ['称', 'v', (2054, 'COO')], ['说', 'v', (2099, 'SBV')], ['称', 'v', (2132, 'COO')], ['称', 'v', (2158, 'COO')], ['称', 'v', (2272, 'COO')], ['说', 'v', (3139, 'COO')], ['说', 'v', (3329, 'COO')], ['称', 'v', (3682, 'ATT')], ['称', 'v', (3723, 'COO')], ['称', 'v', (3865, 'VOB')], ['称', 'v', (3884, 'COO')], ['说', 'v', (4155, 'VOB')], ['说', 'v', (4235, 'COO')], ['说', 'v', (4370, 'COO')], ['称', 'v', (4643, 'VOB')], ['说', 'v', (5311, 'COO')], ['说', 'v', (5366, 'COO')], ['称', 'v', (5619, 'VOB')], ['称', 'v', (5720, 'VOB')], ['说', 'v', (6657, 'COO')], ['说', 'v', (6901, 'COO')], ['称', 'v', (6970, 'COO')], ['称', 'v', (7001, 'COO')], ['指出', 'v', (7060, 'COO')], ['称', 'v', (7210, 'VOB')], ['称', 'v', (7276, 'COO')], ['说', 'v', (8651, 'VOB')], ['说'

107

In [54]:
' '.join(word_list[67:229])

'透视 百色 说 脱贫 在 广西 百色市 右江区 四塘镇 永靖村 村民 驾驶 甘蔗 种植机 种植 甘蔗 2016年 3月 17日 摄 自然 条件 差 传统 农业 无法 带动 贫困 群众 脱贫 该 怎么 去 打 赢 脱贫 攻坚战 广西 百色 面对 的 困境 具有 典型 意义 然而 仅 去年 一 年 百色 就 有 134 个 贫困村 脱贫 出 列 这 是 如何 实现 的 推动 特色 产业 发展 增强 贫困 地区 自我 发展 能力 是 今年 政府 工作 报告 关于 扶贫 工作 的 重要 精神 之一 解析 百色 产业 扶贫 实践 为 老区 脱贫 攻坚 提供 了 一个 可 供 借鉴 的 样本 新华社 记者 黄孝邦 摄 新华社 长沙 4月 12日 电 记者 阳建 11日 中 车 株洲 电力 机车 有限公司 与 马来西亚 交通部 在 长沙 签订 两 款 米轨 1 米 宽轨 距 动车组 购销 合同 包括 13 列 混合 动力 电动车组 和 9 列 现代 超级 动车组 这 是 自 2014年 以来 中国 再次 获得 马来西亚 的 米轨 动车组 订单'

In [55]:
structure_summary[67:229]

[['透视', 'v', (60, 'COO')],
 ['百色', 'ns', (68, 'VOB')],
 ['说', 'v', (68, 'COO')],
 ['脱贫', 'v', (70, 'VOB')],
 ['在', 'p', (79, 'ADV')],
 ['广西', 'ns', (74, 'ATT')],
 ['百色市', 'ns', (75, 'ATT')],
 ['右江区', 'ns', (76, 'ATT')],
 ['四塘镇', 'ns', (77, 'ATT')],
 ['永靖村', 'ns', (78, 'ATT')],
 ['村民', 'n', (72, 'POB')],
 ['驾驶', 'v', (71, 'COO')],
 ['甘蔗', 'n', (81, 'ATT')],
 ['种植机', 'n', (79, 'VOB')],
 ['种植', 'v', (79, 'COO')],
 ['甘蔗', 'n', (82, 'VOB')],
 ['2016年', 'nt', (85, 'ATT')],
 ['3月', 'nt', (86, 'ATT')],
 ['17日', 'nt', (87, 'ADV')],
 ['摄', 'v', (82, 'COO')],
 ['自然', 'n', (89, 'ATT')],
 ['条件', 'n', (92, 'ATT')],
 ['差', 'a', (92, 'ATT')],
 ['传统', 'n', (92, 'ATT')],
 ['农业', 'n', (94, 'SBV')],
 ['无法', 'v', (94, 'ADV')],
 ['带动', 'v', (87, 'COO')],
 ['贫困', 'a', (96, 'ATT')],
 ['群众', 'n', (94, 'DBL')],
 ['脱贫', 'v', (94, 'VOB')],
 ['该', 'v', (101, 'ADV')],
 ['怎么', 'r', (101, 'ADV')],
 ['去', 'v', (101, 'ADV')],
 ['打', 'v', (97, 'COO')],
 ['赢', 'v', (104, 'ATT')],
 ['脱贫', 'v', (104, 'ATT')],
 ['攻坚战', 'n',

In [59]:
for i in range(len(say_structure_summary)):
    if i < len(say_structure_summary) - 1:
        start_site = say_structure_summary[i][2][0] - 1
        end_site = say_structure_summary[i+1][2][0] - 1
    else:
        start_site = say_structure_summary[i][2][0] - 1
        end_site = len(structure_summary)
        
    print(' '.join(word_list[start_site:end_site] + ['\n'])) 

透视 百色 说 脱贫 在 广西 百色市 右江区 四塘镇 永靖村 村民 驾驶 甘蔗 种植机 种植 甘蔗 2016年 3月 17日 摄 自然 条件 差 传统 农业 无法 带动 贫困 群众 脱贫 该 怎么 去 打 赢 脱贫 攻坚战 广西 百色 面对 的 困境 具有 典型 意义 然而 仅 去年 一 年 百色 就 有 134 个 贫困村 脱贫 出 列 这 是 如何 实现 的 推动 特色 产业 发展 增强 贫困 地区 自我 发展 能力 是 今年 政府 工作 报告 关于 扶贫 工作 的 重要 精神 之一 解析 百色 产业 扶贫 实践 为 老区 脱贫 攻坚 提供 了 一个 可 供 借鉴 的 样本 新华社 记者 黄孝邦 摄 新华社 长沙 4月 12日 电 记者 阳建 11日 中 车 株洲 电力 机车 有限公司 与 马来西亚 交通部 在 长沙 签订 两 款 米轨 1 米 宽轨 距 动车组 购销 合同 包括 13 列 混合 动力 电动车组 和 9 列 现代 超级 动车组 这 是 自 2014年 以来 中国 再次 获得 马来西亚 的 米轨 动车组 订单 

据 中 车 株洲 电力 机车 有限公司 动车组 研发部 部长 周安德 介绍 13 列 混合 动力 电动车组 为 电力 超级 电容 结构 属于 一 种 技术 创 新型 动车组 列车 牵引 时 60000 法 拉 超级 电容器 可以 短 时间 提供 大功率 电流 供 列车 启动 加速 列车 制动 时 超过 85 的 制动 能量 可以 被 超级 电容 吸收 存储 供 列车 下次 启动 使用 从而 实现 能量 循环 利用 减少 能源 消耗 该 列车 运行 时速 100 公里 四节 编组 载客量 达 500 余 人 另外 9列 现代 超级 动车组 则 是 2014年 中 车 株洲 电力 机车 有限公司 出口 马来西亚 世界 最高 速米 轨 动车组 的 增购 列车 该 款 动车 在 米轨 上 每 小时 运行 速度 可 达 160 公里 并 在 原来 基础 上 做 了 全新 技术 升级 信息化 智能化 程度 更 高 相比 标准 轨道 动车组 米轨 动车组 研制 难度 更 大 因为 轨道 越 窄 对 列车 运行 的 平稳性 走行 部分 转向架 和 车体 强度 设计 要求 更 高 目前 中车 株洲 电力 机车 有限公司 正 全力 推动 海外 各 子公司 